In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: ai-tci-ml
Azure region: westeurope
Subscription id: 7c03dd83-6b95-43b1-9f53-23dfd07e8803
Resource group: azp-078-rg


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
cluster_name = "UdacityStandardD2V2"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_cluster = ComputeTarget(workspace=ws,name=cluster_name)
    print("cluster exists")
except ComputeTargetException:
    amlcompute_Config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_v2",max_nodes=4)
    compute_cluster = ComputeTarget.create(workspace=ws,name=cluster_name,provisioning_configuration=amlcompute_Config)
    compute_cluster.wait_for_completion(show_output=True)


InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
# 
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--max_iter': choice(10,20,30,40,50,75,100),
    '--C': uniform(0.0,2.0)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=3,slack_factor=0.5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./',script='train.py',compute_target=compute_cluster,environment=sklearn_env,)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,hyperparameter_sampling=ps,policy=policy,primary_metric_name='Accuracy',
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=10,max_concurrent_runs=2)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
experiment = Experiment(workspace=ws, name='udacity_project1')
run = experiment.submit(config=hyperdrive_config)
run.wait_for_completion(show_output=True)

RunId: HD_ed058768-fa01-47f5-9f89-b3050ef2f714
Web View: https://ml.azure.com/runs/HD_ed058768-fa01-47f5-9f89-b3050ef2f714?wsid=/subscriptions/7c03dd83-6b95-43b1-9f53-23dfd07e8803/resourcegroups/azp-078-rg/workspaces/ai-tci-ml&tid=eb70b763-b6d7-4486-8555-8831709a784e

Streaming azureml-logs/hyperdrive.txt

[2023-08-23T14:10:08.615378][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space
[2023-08-23T14:10:09.4713225Z][SCHEDULER][INFO]Scheduling job, id='HD_ed058768-fa01-47f5-9f89-b3050ef2f714_0' 
[2023-08-23T14:10:09.5469472Z][SCHEDULER][INFO]Scheduling job, id='HD_ed058768-fa01-47f5-9f89-b3050ef2f714_1' 
[2023-08-23T14:10:09.497046][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.
[2023-08-23T14:10:10.1789636Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_ed058768-fa01-47f5-9f89-b3050ef2f714_1' 
[2023-08-23T14:10:10.1637724Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_ed058768-fa01-47f5-9f89-b3050ef

{'runId': 'HD_ed058768-fa01-47f5-9f89-b3050ef2f714',
 'target': 'UdacityStandardD2V2',
 'status': 'Completed',
 'startTimeUtc': '2023-08-23T14:10:07.741557Z',
 'endTimeUtc': '2023-08-23T14:33:20.294791Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1b072f36-88b7-42ed-b196-9c794b9b0a0f',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': 'infinite_space_size',
  'score': '0.9106579266812334',
  'best_child_run_id': 'HD_ed058768-fa01-47f5-9f89-b3050ef2f714_8',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_ed058768-fa01-47f5-9f89-b3050ef2f714_8'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telem

In [7]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_details = best_run.get_details()['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',best_run_details)

Best Run Id:  HD_ed058768-fa01-47f5-9f89-b3050ef2f714_8
 -Accuracy: 0.9106579266812334
 -Arguments: ['--C', '0.34835900599541003', '--max_iter', '50']


In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [7]:
# best_run.#saving the best model
from azureml.core.model import Model
best_run.register_model(model_path='outputs/project1model.pkl',model_name='sai_hyp_tune_model_udacity',properties={'Accuracy':best_run_metrics['Accuracy']})

Model(workspace=Workspace.create(name='ai-tci-ml', subscription_id='7c03dd83-6b95-43b1-9f53-23dfd07e8803', resource_group='azp-078-rg'), name=sai_hyp_tune_model_udacity, id=sai_hyp_tune_model_udacity:1, version=1, tags={}, properties={'Accuracy': '0.9098082058752124'})

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data = TabularDatasetFactory().from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [5]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)
x_train, x_test, y_train,_y_test = train_test_split(x,y,test_size=0.25,shuffle=True)
train_set = pd.concat([x_train, y_train], axis=1, join='inner')
test_set = pd.concat([x_test, _y_test], axis=1, join='inner')

In [29]:
from azureml.core import Dataset, Datastore
ds = ws.get_default_datastore()
train_data = Dataset.Tabular.register_pandas_dataframe(dataframe=train_set,name='udacity_automl_project_traindata',target=ds)
test_data = Dataset.Tabular.register_pandas_dataframe(dataframe=test_set,name='udacity_automl_project_testdata',target=ds)
# test_data = Dataset.from_pandas_dataframe(test_set)
# tabular_dataset = tabular_dataset.register(workspace=workspace, name='my_tabular_dataset', description='My Tabular Dataset')

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/b153c645-5f6a-4d5e-b373-3e0fb55de522/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [30]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    name='Udacity-AutoML-project',
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    validation_data=test_data,
    label_column_name='y',
    max_concurrent_iterations=2,
    iterations=10,
    compute_target=compute_cluster
    )

In [31]:
# Submit your automl run

automl_exp = Experiment(workspace=ws,name='AutoML-Udactity-project')
automl_run = automl_exp.submit(automl_config)
automl_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-Udactity-project,AutoML_0bc71f89-579c-43be-bf60-131cc3f85310,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-Udactity-project,AutoML_0bc71f89-579c-43be-bf60-131cc3f85310,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in th

{'runId': 'AutoML_0bc71f89-579c-43be-bf60-131cc3f85310',
 'target': 'UdacityStandardD2V2',
 'status': 'Completed',
 'startTimeUtc': '2023-08-23T09:39:14.504174Z',
 'endTimeUtc': '2023-08-23T10:11:26.877117Z',
 'services': {},
 'properties': {'num_iterations': '10',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'UdacityStandardD2V2',
  'AMLSettingsJsonString': '{"path":null,"name":"Udacity-AutoML-project","subscription_id":"7c03dd83-6b95-43b1-9f53-23dfd07e8803","resource_group":"azp-078-rg","workspace_name":"ai-tci-ml","region":"westeurope","compute_target":"UdacityStandardD2V2","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_parallel_run":false,"num_procs":null,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_ge

In [33]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_best_run = automl_run.get_best_child()


In [34]:
automl_best_run_metrics = automl_best_run.get_metrics()

In [36]:
automl_best_run.register_model(model_path='outputs/model.pkl', model_name='udacity_automl_model',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': automl_best_run_metrics['AUC_weighted'], 'Accuracy': automl_best_run_metrics['accuracy']})


Model(workspace=Workspace.create(name='ai-tci-ml', subscription_id='7c03dd83-6b95-43b1-9f53-23dfd07e8803', resource_group='azp-078-rg'), name=udacity_automl_model, id=udacity_automl_model:1, version=1, tags={'Training context': 'Auto ML'}, properties={'AUC': '0.93915770609319', 'Accuracy': '0.9163631949502307'})

In [5]:
compute_cluster.delete()